# Questions 

## How many hidden layers are used?
##### 1

## Which activation function should be used in the hidden layer(s)?
#### none linear e.g. ReLU, sigmuid  

## Which activation function should be used in the output layer?
#### any or nothing if the softmay layer is used afterwards

## Which loss function is used?
#### cross-entropy

# First Experimentation

In [ ]:
from sklearn.datasets import load_breast_cancer
import numpy as np
x,y = load_breast_cancer(return_X_y = True, as_frame = True)
x.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

x_train, x_test, y_train, y_test = train_test_split(x ,y, train_size=0.9, random_state=42)

scaler = StandardScaler(copy=True)
xTrain_scaled = scaler.fit_transform(x_train, y_train)
xTest_scaled = scaler.transform(x_test)

clf = MLPClassifier(random_state=42, hidden_layer_sizes=(64),learning_rate_init=0.0001, batch_size=100, activation='relu')
#clf.out_activation_ can not be overwritten, would change after fit again to logistic
#clf.out_activation_='softmax'
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print(clf.score(x_test,y_test))
#print(clf.out_activation_)
#print(clf.n_outputs_)
print(classification_report(y_test, y_pred))
plot_confusion_matrix(clf, x_test, y_test, cmap ='Blues')

#### Train the MLPClassifier with SGD optimizer and the default L2 regularization. No momentum (needs to be turned off).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


def plot_learning_curve(df_loss_curve, title='Loss curve - Breast Cancer Data'):
    sns.lineplot(data=df_loss_curve, dashes=False)
    plt.title(title)
    plt.show()

max_iter = 200
x_train, x_test, y_train, y_test = train_test_split(x ,y, train_size=0.9, random_state=42)

scaler = StandardScaler(copy=True)
xTrain_scaled = scaler.fit_transform(x_train, y_train)
xTest_scaled = scaler.transform(x_test)

clf = MLPClassifier(random_state=42, max_iter= max_iter, hidden_layer_sizes=[64],learning_rate_init=0.01, batch_size=100, activation='relu', solver = "sgd")
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print(clf.score(x_test,y_test))
print(classification_report(y_test, y_pred))
#plot_confusion_matrix(clf, x_test, y_test, cmap ='Blues')

loss_curve = clf.loss_curve_ + [np.NaN] * (max_iter - len(clf.loss_curve_))
loss_curve_df = pd.DataFrame(loss_curve, columns=['SGD Baseline'])
plot_learning_curve(loss_curve_df)




In [ ]:
clf.early_stopping = True